In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
#import ipynb.fs.full.config as cfg
import pathlib

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

class gmail_scrape(object):
    def __init__(self):
    #Shows basic usage of the Gmail API.
    #Lists the user's Gmail labels.
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file('client_secret_gmail.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        self.service = build('gmail', 'v1', credentials=creds)

    
    def search(self,search_term):
        # Call the Gmail API
        results = self.service.users().messages().list(
            userId='me', 
            # labelIds = ['INBOX'],
            q=search_term,
            ).execute()
        messages = results.get('messages', [])

        #getting authenticated user's profile to fetch email address
        profile = self.service.users().getProfile(userId='me').execute()
        f = open("search results.html", "a")
        f.write("<h3>Gmail:</h3>\n")
        if not messages:
            f.write('No emails found.'+ '\n')
        else:
            
            f.write("<ol>\n")
            for message in messages:
                msg = self.service.users().messages().get(
                    userId='me', 
                    id=message['id']).execute()
                # print(msg.get("id"))
                email_link = "https://mail.google.com/mail?authuser="+profile['emailAddress']+"#all/"+msg.get("id")
                f.write("<li> <a href=\""+email_link+"\">")
                for name in msg['payload']['headers']:
                    if(name["name"] == "Subject"):
                        f.write(name["value"])
                f.write("</a></li>\n")
        f.write("</ol>\n")       
        f.close()
        return messages
